In [1]:
import pandas as pd
import requests, json
import numpy as np
from datetime import datetime
import os
wd = '/home/beeb/Insync/sei2112@columbia.edu/Google Drive/Columbia SusDev PhD/Research/My work/predictArsenic/'
os.chdir(wd)
datInpath = 'data/raw/bamwsp/bamwsp_wSonargaon_avg.csv'
keyInpath = 'data/raw/googlePlaces/apiKey.txt'
usageInpath = 'data/intermediate/googlePlaces/usage.csv'
outpath = 'data/raw/googlePlaces/{}.txt'
errorsName = 'errors'
resultsName = 'results'
# url variable store url 
url = "https://maps.googleapis.com/maps/api/geocode/json?address={query}&key={key}"
MAXUSAGEMONTHLY = 30000
MAXUSAGEDAILY = 1333

In [2]:
# set up API key
api_key = pd.read_csv(keyInpath, header = None, squeeze = True)[0]

# Read in BAMWSP data, create a list of queries to send to Google
The data consist of a list of Bangladeshi villages, the number of wells in each one, and the proportion of those that are 'safe'. The geographics given are District, Upazila, Union, Mouza and Village. However, it is raw for Google to correctly geocode a Bangladeshi village, so instead we create a query at the Mouza level. This leaves us with ~22,600 unique obs.

In [3]:
# read in BAMWSP village data and create list of queries
dat = pd.read_csv(datInpath)
dat['query'] = dat['Mouza'] + ',+' + dat['Union']+ ',+Bangladesh'
queries = dat['query'].unique()

In [4]:
## PREPARE OUTPUT - successful and unsuccessful attempts at geocoding

# if we already have results, load them and exclude already-run queries; else create a new empty list
if os.path.exists(outpath.format(resultsName)):
    with open(outpath.format(resultsName)) as f:
        results = json.load(f)
    queries = [i for i in queries if i not in results.keys()]
else:
    results = dict()

    
# do the same for errors
if os.path.exists(outpath.format(errorsName)):
    with open(outpath.format(errorsName)) as f:
        errors = json.load(f)
    queries = [i for i in queries if i not in errors]   
else:
    errors = []


# Usage handling
Google charges if you use their services too much. Therefore, it's important to limit your monthly usage. If you use more than 40k hits on the Geocode API you start getting charged 5USD per 1000 hits, so this bit of code makes sure that I'm using less than that.

In [5]:
today = datetime.today()
if len(str(today.month)) == 1:
    m = '0' + str(today.month)
month = int(str(today.year) + m)

# check if we already have usage stats, if not create a new one
if os.path.exists(usageInpath):
    usage = pd.read_csv(usageInpath)
else:
    usage = pd.DataFrame(data = {'month' : [month], 'hits' : [0]})

# check if we have any usage for the current month, if not add it in
if max(usage.month) < month:
    newUsage = pd.DataFrame(data = {'month' : [month], 'hits' : [0]})
    usage = pd.concat([usage, newUsage])
    
currentMonthlyUsage = usage.hits[usage.month == month].iloc[0]
currentDailyUsage = 0

In [6]:

for query in queries[(len(results) + len(errors)):len(queries)]:
    # only query if we're not past max usage per day/month
    if (currentMonthlyUsage < MAXUSAGEMONTHLY) and (currentDailyUsage < MAXUSAGEDAILY):
        
        # keep track of how many iterations we've had
        # print the current status
        # save the current # of queries carried out
        if currentMonthlyUsage % 100 == 0:
            print('Iteration ' + str(currentDailyUsage) + ' at ' 
                      + str(datetime.now()) + ' for query ' + query)
            usage.hits[usage.month == month] = currentMonthlyUsage
            usage.to_csv(usageInpath, index = False)
        # start iterating through remaining 
        try:

            # get method of requests module 
            # return response object 
            r = requests.get(url.format(query=query, key = api_key)) 

            # json method of response object convert 
            #  json format data into python format data 
            x = r.json()
            if x['status'] == 'OK':
                x['query'] = query
                results[query] = x
            else:
                errors.append(query)
        except:
            errors.append(query)

        currentMonthlyUsage += 1
        currentDailyUsage += 1
# record the extra usage that took place today        
usage.hits[usage.month == month] = currentMonthlyUsage
usage.to_csv(usageInpath, index = False)

# save the new results
with open(outpath.format(resultsName), 'w') as f:
    json.dump(results, f)

Iteration 1400 at 2021-01-22 05:27:44.891948 for query Paglirbil,+Haldia Palong,+Bangladesh
Iteration 1500 at 2021-01-22 05:27:53.833498 for query PURATAN BANDURA,+Bandura,+Bangladesh
Iteration 1600 at 2021-01-22 05:28:02.644145 for query SHIKARI PARA,+Shikaripara,+Bangladesh
Iteration 1700 at 2021-01-22 05:28:10.857709 for query Jhalijhali,+Palashbari,+Bangladesh
Iteration 1800 at 2021-01-22 05:28:18.897622 for query Mithapur,+Alfadanga,+Bangladesh
Iteration 1900 at 2021-01-22 05:28:26.948921 for query Unknown (JI),+Chandra,+Bangladesh
Iteration 2000 at 2021-01-22 05:28:34.431341 for query Vimpur,+Ghoshpur,+Bangladesh
Iteration 2100 at 2021-01-22 05:28:42.780231 for query Sahasrail,+Shekhar,+Bangladesh
Iteration 2200 at 2021-01-22 05:28:51.118047 for query Akain,+Kaijuri,+Bangladesh
Iteration 2300 at 2021-01-22 05:28:59.825547 for query Sidhlajuri,+Dumain,+Bangladesh
Iteration 2400 at 2021-01-22 05:29:07.582909 for query Majhkandi,+Raipur,+Bangladesh
Iteration 2500 at 2021-01-22 05:29

In [9]:
usage.hits[usage.month == month] = currentMonthlyUsage
usage.to_csv(usageInpath, index = False)

In [10]:
len(errors)

1388

In [11]:
len(results)

2559